In [1]:
import numpy as np
import pickle
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, CuDNNLSTM, TimeDistributed, \
                         Bidirectional, Flatten, \
                         Embedding, Dropout, Flatten, BatchNormalization, \
                         RNN, concatenate, Activation, LSTM
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from keras.layers.convolutional import Conv2D
import random as rn
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
import codecs
from tensorflow.contrib.rnn import *

Using TensorFlow backend.


In [2]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [3]:
# load feature and labels
feat = np.load('../../dimensionalSM/data/feat_34_hfs.npy')
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')
# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)

# standardization
scaled_feature = False

In [4]:
import pickle
with open('../../dimensionalSM/IEMOCAP_full_release/data_collected_333.pickle', 'rb') as handle:
    data2 = pickle.load(handle)
x_train_mocap = []
counter = 0
for ses_mod in data2:
    x_head = ses_mod['mocap_head']
    if(x_head.shape != (200,18)):
        x_head = np.zeros((200,18))   
    x_head[np.isnan(x_head)]=0
    x_hand = ses_mod['mocap_hand']
    if(x_hand.shape != (200,6)):
        x_hand = np.zeros((200,6))   
    x_hand[np.isnan(x_hand)]=0
    x_rot = ses_mod['mocap_rot']
    if(x_rot.shape != (200,165)):
        x_rot = np.zeros((200,165))   
    x_rot[np.isnan(x_rot)]=0
    x_mocap = np.concatenate((x_head, x_hand), axis=1)
    x_mocap = np.concatenate((x_mocap, x_rot), axis=1)
    x_train_mocap.append( x_mocap )
    
x_train_mocap = np.array(x_train_mocap)
x_train_mocap = x_train_mocap.reshape(-1,200,189,1)
x_train_mocap.shape

(10039, 200, 189, 1)

In [5]:
if scaled_feature == True:
    scaler = StandardScaler()
    scaler = scaler.fit(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaler.transform(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaled_feat.reshape(feat.shape[0], feat.shape[1], feat.shape[2])
    feat = scaled_feat
else:
    feat = feat

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1, keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss


In [6]:
# API model
def model():
    # speech network
    input_speech = Input(shape=(feat.shape[1], feat.shape[2]), name='speech_input')
    net_speech = BatchNormalization()(input_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(128, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    model_speech = Flatten()(net_speech)
    #model_speech = Dropout(0.1)(net_speech)
    
#     #text network
#     input_text = Input(shape=(MAX_SEQUENCE_LENGTH, ))
#     net_text = Embedding(nb_words,
#                     EMBEDDING_DIM,
#                     weights = [g_word_embedding_matrix],
#                     trainable = True)(input_text)
#     net = Bidirectional(LSTM(128, return_sequences=True))(net_text)
#     net = Bidirectional(LSTM(64, return_sequences=False))(net)  
#     model_text = Dense(64)(net)
    
    #mocap network
    input_mocap = Input(shape=(200,189,1))
    model_mocap0 = BatchNormalization()(input_mocap)
    model_mocap1 = Conv2D(64, 3, strides=(3, 3), padding="same")(model_mocap0)
    model_mocap2 = Dropout(0.2)(model_mocap1)
    model_mocap3 = Activation('relu')(model_mocap2)
    model_mocap4 = Conv2D(128, 3,strides=(3, 3), padding="same")(model_mocap3)
    model_mocap5 = Dropout(0.2)(model_mocap4)
    model_mocap6 = Activation('relu')(model_mocap5)
    model_mocap7 = Conv2D(256, 3,strides=(3, 3), padding="same")(model_mocap6)
    model_mocap8 = Dropout(0.2)(model_mocap7) 
    model_mocap9 = Activation('relu')(model_mocap8)
    model_mocap10 = Conv2D(128, 3,strides=(3, 3), padding="same")(model_mocap9)   
    model_mocap11 = Dropout(0.2)(model_mocap10)  
    model_mocap12 = Activation('relu')(model_mocap11)
    model_mocap13 = Conv2D(64, 3,strides=(3, 3), padding="same")(model_mocap12)  
    model_mocap14 = Dropout(0.2)(model_mocap13) 
    model_mocap15 = Activation('relu')(model_mocap14)
    model_mocap = Flatten()(model_mocap15) 
    #model_mocap = Dropout(dropout_rate)(model_mocap16)
    # combined model
    model_combined = concatenate([model_speech, model_mocap])
    model_combined = Dense(64, activation='relu')(model_combined)
    model_combined = Dense(32, activation='relu')(model_combined)
    #model_combined = Dropout(0.4)(model_combined)
    target_names = ('v', 'a', 'd')
    model_combined = [Dense(1, name=name)(model_combined) for name in target_names]

    model = Model([input_speech, input_mocap], model_combined) 
    model.compile(loss=ccc_loss,
                  loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model


# model_1 = model()
# model_1.summary()

In [7]:
# #speech_to_npy
# model1 = model()
# earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
#                               restore_best_weights=True)
# hist1 = model1.fit([feat[:8000],x_train_mocap[:8000]], vad[:8000].T.tolist(), epochs=50, 
#                    batch_size=32, verbose=1, validation_split=0.2, shuffle=True,
#                    callbacks=[earlystop])

# eval_metrik1 = model1.evaluate([feat[8000:],x_train_mocap[8000:]], vad[8000:].T.tolist())
# print(eval_metrik1)

# # make prediction
# predict = model1.predict([feat[6400:],x_train_mocap[6400:]], batch_size=32)
# np.save('speech_mocap_npy-batch8000-2', 
#          np.array(predict).reshape(3, 3639).T)


In [8]:
# main function for for LOSO
def main(time):
    model_1 = model()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [feat[:8000],x_train_mocap[:8000]],
                      vad[:8000].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True, callbacks=[earlystop]
                      )
    metrik = model_1.evaluate( [feat[8000:],x_train_mocap[8000:]], vad[8000:].T.tolist())
    print(metrik[-3:])
    # make prediction
    predict = model_1.predict([feat[6400:],x_train_mocap[6400:]], batch_size=32)
    np.save('npy8000/speech_mocap_npy-8000batch'+str(time)+'.npy', 
             np.array(predict).reshape(3, 3639).T)
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(20):
        best = main(time)
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('npy8000/sm8000-3-batch-again1.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 6400 samples, validate on 1600 samples
Epoch 1/50
6400/6400 [==============================] - 56s 9ms/step - loss: 1.8701 - v_loss: 0.7633 - a_loss: 0.4995 - d_loss: 0.6073 - v_ccc: 0.2367 - a_ccc: 0.5005 - d_ccc: 0.3927 - val_loss: 2.4714 - val_v_loss: 0.8396 - val_a_loss: 0.8194 - val_d_loss: 0.8124 - val_v_ccc: 0.1604 - val_a_ccc: 0.1806 - val_d_ccc: 0.1876
Epoch 2/50
6400/6400 [==============================] - 52s 8ms/step - loss: 1.2714 - v_loss: 0.4580 - a_loss: 0.3490 - d_loss: 0.4644 - v_ccc: 0.5420 - a_ccc: 0.6510 - d_ccc: 0.5356 - val_loss: 2.0592 - val_v_loss: 0.7946 - val_a_loss: 0.5810 - val_d_loss: 0.6836 - val_v_ccc: 0.2054 - val_a_ccc: 0.4190 - val_d_ccc: 0.3164
Epoch 3/50
6400/6400 [==============================] - 51s 8ms/step - loss: 1.1938 - v_loss: 0.4080 - a_loss: 0.3383 - d_loss: 0.4476 - v_ccc: 0.5920 - a_ccc: 0.6617 - d_ccc: 0.5524 - val_loss: 1.8646 - val_v_loss: 0.79

Epoch 10/50
6400/6400 [==============================] - 50s 8ms/step - loss: 1.0245 - v_loss: 0.3304 - a_loss: 0.3071 - d_loss: 0.3871 - v_ccc: 0.6696 - a_ccc: 0.6929 - d_ccc: 0.6129 - val_loss: 1.9239 - val_v_loss: 0.8266 - val_a_loss: 0.4650 - val_d_loss: 0.6323 - val_v_ccc: 0.1734 - val_a_ccc: 0.5350 - val_d_ccc: 0.3677
Epoch 11/50
6400/6400 [==============================] - 50s 8ms/step - loss: 1.0088 - v_loss: 0.3295 - a_loss: 0.3000 - d_loss: 0.3792 - v_ccc: 0.6705 - a_ccc: 0.7000 - d_ccc: 0.6208 - val_loss: 1.9557 - val_v_loss: 0.8130 - val_a_loss: 0.5036 - val_d_loss: 0.6391 - val_v_ccc: 0.1870 - val_a_ccc: 0.4964 - val_d_ccc: 0.3609
Epoch 12/50
6400/6400 [==============================] - 51s 8ms/step - loss: 0.9992 - v_loss: 0.3231 - a_loss: 0.2984 - d_loss: 0.3778 - v_ccc: 0.6769 - a_ccc: 0.7016 - d_ccc: 0.6222 - val_loss: 1.8261 - val_v_loss: 0.7614 - val_a_loss: 0.4701 - val_d_loss: 0.5945 - val_v_ccc: 0.2386 - val_a_ccc: 0.5299 - val_d_ccc: 0.4055
Epoch 13/50
6400/6400 

Epoch 21/50
6400/6400 [==============================] - 51s 8ms/step - loss: 0.9178 - v_loss: 0.2928 - a_loss: 0.2791 - d_loss: 0.3459 - v_ccc: 0.7072 - a_ccc: 0.7209 - d_ccc: 0.6541 - val_loss: 1.7852 - val_v_loss: 0.7245 - val_a_loss: 0.4634 - val_d_loss: 0.5972 - val_v_ccc: 0.2755 - val_a_ccc: 0.5366 - val_d_ccc: 0.4028
Epoch 22/50
6400/6400 [==============================] - 53s 8ms/step - loss: 0.9058 - v_loss: 0.2915 - a_loss: 0.2776 - d_loss: 0.3367 - v_ccc: 0.7085 - a_ccc: 0.7224 - d_ccc: 0.6633 - val_loss: 1.8047 - val_v_loss: 0.7223 - val_a_loss: 0.4708 - val_d_loss: 0.6115 - val_v_ccc: 0.2777 - val_a_ccc: 0.5292 - val_d_ccc: 0.3885
Epoch 23/50
6400/6400 [==============================] - 51s 8ms/step - loss: 0.8943 - v_loss: 0.2882 - a_loss: 0.2735 - d_loss: 0.3326 - v_ccc: 0.7118 - a_ccc: 0.7265 - d_ccc: 0.6674 - val_loss: 1.8257 - val_v_loss: 0.7401 - val_a_loss: 0.4809 - val_d_loss: 0.6047 - val_v_ccc: 0.2599 - val_a_ccc: 0.5191 - val_d_ccc: 0.3953
Epoch 24/50
6400/6400 

2039/2039 [==============================] - 2s 1ms/step
[0.4015158414840698, 0.5957627892494202, 0.4323948919773102]
3
Train on 6400 samples, validate on 1600 samples
Epoch 1/50
6400/6400 [==============================] - 54s 8ms/step - loss: 1.9012 - v_loss: 0.8016 - a_loss: 0.4855 - d_loss: 0.6141 - v_ccc: 0.1984 - a_ccc: 0.5145 - d_ccc: 0.3859 - val_loss: 2.4395 - val_v_loss: 0.8811 - val_a_loss: 0.7342 - val_d_loss: 0.8243 - val_v_ccc: 0.1189 - val_a_ccc: 0.2658 - val_d_ccc: 0.1757
Epoch 2/50
6400/6400 [==============================] - 51s 8ms/step - loss: 1.2729 - v_loss: 0.4710 - a_loss: 0.3453 - d_loss: 0.4566 - v_ccc: 0.5290 - a_ccc: 0.6547 - d_ccc: 0.5434 - val_loss: 1.8733 - val_v_loss: 0.7682 - val_a_loss: 0.4881 - val_d_loss: 0.6170 - val_v_ccc: 0.2318 - val_a_ccc: 0.5119 - val_d_ccc: 0.3830
Epoch 3/50
6400/6400 [==============================] - 52s 8ms/step - loss: 1.1956 - v_loss: 0.4093 - a_loss: 0.3380 - d_loss: 0.4484 - v_ccc: 0.5907 - a_ccc: 0.6620 - d_ccc: 0.5516

6400/6400 [==============================] - 45s 7ms/step - loss: 1.1193 - v_loss: 0.3676 - a_loss: 0.3272 - d_loss: 0.4244 - v_ccc: 0.6324 - a_ccc: 0.6728 - d_ccc: 0.5756 - val_loss: 1.7768 - val_v_loss: 0.7527 - val_a_loss: 0.4589 - val_d_loss: 0.5652 - val_v_ccc: 0.2473 - val_a_ccc: 0.5411 - val_d_ccc: 0.4348
Epoch 6/50
6400/6400 [==============================] - 46s 7ms/step - loss: 1.0888 - v_loss: 0.3575 - a_loss: 0.3197 - d_loss: 0.4116 - v_ccc: 0.6425 - a_ccc: 0.6803 - d_ccc: 0.5884 - val_loss: 1.7334 - val_v_loss: 0.7257 - val_a_loss: 0.4439 - val_d_loss: 0.5638 - val_v_ccc: 0.2743 - val_a_ccc: 0.5561 - val_d_ccc: 0.4362
Epoch 7/50
6400/6400 [==============================] - 46s 7ms/step - loss: 1.0715 - v_loss: 0.3503 - a_loss: 0.3146 - d_loss: 0.4066 - v_ccc: 0.6497 - a_ccc: 0.6854 - d_ccc: 0.5934 - val_loss: 1.7869 - val_v_loss: 0.7808 - val_a_loss: 0.4473 - val_d_loss: 0.5589 - val_v_ccc: 0.2192 - val_a_ccc: 0.5527 - val_d_ccc: 0.4411
Epoch 8/50
6400/6400 [==============

...

Epoch 15/50
6400/6400 [==============================] - 47s 7ms/step - loss: 0.9571 - v_loss: 0.3159 - a_loss: 0.2854 - d_loss: 0.3557 - v_ccc: 0.6841 - a_ccc: 0.7146 - d_ccc: 0.6443 - val_loss: 1.8810 - val_v_loss: 0.7272 - val_a_loss: 0.5139 - val_d_loss: 0.6399 - val_v_ccc: 0.2728 - val_a_ccc: 0.4861 - val_d_ccc: 0.3601
Epoch 16/50
2039/2039 [==============================] - 2s 1ms/step
[0.38666287064552307, 0.5809705257415771, 0.41406741738319397]
14
Train on 6400 samples, validate on 1600 samples
Epoch 1/50
6400/6400 [==============================] - 57s 9ms/step - loss: 1.9261 - v_loss: 0.8532 - a_loss: 0.4614 - d_loss: 0.6115 - v_ccc: 0.1468 - a_ccc: 0.5386 - d_ccc: 0.3885 - val_loss: 2.3833 - val_v_loss: 0.8453 - val_a_loss: 0.7772 - val_d_loss: 0.7608 - val_v_ccc: 0.1547 - val_a_ccc: 0.2228 - val_d_ccc: 0.2392
Epoch 2/50
6400/6400 [==============================] - 46s 7ms/step - loss: 1.3017 - v_loss: 0.4833 - a_loss: 0.3519 - d_loss: 0.4664 - v_ccc: 0.5167 - a_ccc: 0.6481

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6400/6400 [==============================] - 51s 8ms/step - loss: 1.0455 - v_loss: 0.3387 - a_loss: 0.3111 - d_loss: 0.3957 - v_ccc: 0.6613 - a_ccc: 0.6889 - d_ccc: 0.6043 - val_loss: 1.7920 - val_v_loss: 0.7045 - val_a_loss: 0.4745 - val_d_loss: 0.6131 - val_v_ccc: 0.2955 - val_a_ccc: 0.5255 - val_d_ccc: 0.3869
Epoch 11/50
6400/6400 [==============================] - 52s 8ms/step - loss: 1.0159 - v_loss: 0.3290 - a_loss: 0.3003 - d_loss: 0.3866 - v_ccc: 0.6710 - a_ccc: 0.6997 - d_ccc: 0.6134 - val_loss: 1.7775 - val_v_loss: 0.6943 - val_a_loss: 0.4731 - val_d_loss: 0.6100 - val_v_ccc: 0.3057 - val_a_ccc: 0.5269 - val_d_ccc: 0.3900
Epoch 12/50
6400/6400 [==============================] - 55s 9ms/step - loss: 1.0116 - v_loss: 0.3265 - a_loss: 0.3033 - d_loss: 0.3818 - v_ccc: 0.6735 - a_ccc: 0.6967 - d_ccc: 0.6182 - val_loss: 1.7462 - val_v_loss: 0.6764 - val_a_loss: 0.4761 - val_d_loss: 0.5938 - val_v_ccc: 0.3236 - val_a_ccc: 0.5239 - val_d_ccc: 0.4062
Epoch 13/50
6400/6400 [===========

Epoch 6/50
6400/6400 [==============================] - 57s 9ms/step - loss: 1.1014 - v_loss: 0.3590 - a_loss: 0.3201 - d_loss: 0.4223 - v_ccc: 0.6410 - a_ccc: 0.6799 - d_ccc: 0.5777 - val_loss: 1.8228 - val_v_loss: 0.7332 - val_a_loss: 0.4807 - val_d_loss: 0.6089 - val_v_ccc: 0.2668 - val_a_ccc: 0.5193 - val_d_ccc: 0.3911
Epoch 7/50
6400/6400 [==============================] - 56s 9ms/step - loss: 1.0865 - v_loss: 0.3508 - a_loss: 0.3217 - d_loss: 0.4139 - v_ccc: 0.6492 - a_ccc: 0.6783 - d_ccc: 0.5861 - val_loss: 1.8168 - val_v_loss: 0.7317 - val_a_loss: 0.4861 - val_d_loss: 0.5990 - val_v_ccc: 0.2683 - val_a_ccc: 0.5139 - val_d_ccc: 0.4010
Epoch 8/50
6400/6400 [==============================] - 55s 9ms/step - loss: 1.0615 - v_loss: 0.3432 - a_loss: 0.3141 - d_loss: 0.4042 - v_ccc: 0.6568 - a_ccc: 0.6859 - d_ccc: 0.5958 - val_loss: 1.7899 - val_v_loss: 0.7086 - val_a_loss: 0.4805 - val_d_loss: 0.6009 - val_v_ccc: 0.2914 - val_a_ccc: 0.5195 - val_d_ccc: 0.3991
Epoch 9/50
6400/6400 [===

Epoch 12/50
6400/6400 [==============================] - 47s 7ms/step - loss: 1.0024 - v_loss: 0.3254 - a_loss: 0.2999 - d_loss: 0.3772 - v_ccc: 0.6746 - a_ccc: 0.7001 - d_ccc: 0.6228 - val_loss: 1.7810 - val_v_loss: 0.7270 - val_a_loss: 0.4670 - val_d_loss: 0.5870 - val_v_ccc: 0.2730 - val_a_ccc: 0.5330 - val_d_ccc: 0.4130
Epoch 13/50
6400/6400 [==============================] - 47s 7ms/step - loss: 1.0000 - v_loss: 0.3262 - a_loss: 0.3020 - d_loss: 0.3718 - v_ccc: 0.6738 - a_ccc: 0.6980 - d_ccc: 0.6282 - val_loss: 1.8302 - val_v_loss: 0.7728 - val_a_loss: 0.4696 - val_d_loss: 0.5878 - val_v_ccc: 0.2272 - val_a_ccc: 0.5304 - val_d_ccc: 0.4122
Epoch 14/50
6400/6400 [==============================] - 47s 7ms/step - loss: 0.9970 - v_loss: 0.3179 - a_loss: 0.3000 - d_loss: 0.3792 - v_ccc: 0.6821 - a_ccc: 0.7000 - d_ccc: 0.6208 - val_loss: 1.8147 - val_v_loss: 0.7602 - val_a_loss: 0.4713 - val_d_loss: 0.5833 - val_v_ccc: 0.2398 - val_a_ccc: 0.5287 - val_d_ccc: 0.4167
Epoch 15/50
6400/6400 